This script takes Green Button Data XML (see https://greenbuttondata.org/index.html)
and transforms it into a format usable by SAM for bill savings calculations.

Note that this was written based on data with a 15 minute timestep and at least one full calendar year of data
Future improvements to this script may allow for more flexible data inputs

Also note: during testing we encountered data with PV generation where PV generation steps had a 0 for that timestep (when PV was exporting to the grid)
This script assumes load data does not include a generator that can export to the grid

In [5]:
import certifi
import greenbutton_objects
import greenbutton_objects.parse
import json
import os
import pandas as pd
import PySAM
from PySAM import ResourceTools as tools
import PySAM.Battery as battery_model
import PySAM.Pvsamv1 as pv_model
import PySAM.Utilityrate5 as utility_rate
import PySAM.Cashloan as cashloan
import PySAM.UtilityRateTools
import requests

In [6]:
# Code to extract the relevant data from the interval reading object
# Adapted from https://github.com/JenDobson/greenbutton/tree/master/greenbutton

def parse_reading(interval_reading):
    start = interval_reading.timePeriod.start
    duration = interval_reading.timePeriod.duration
    value = int(interval_reading.value)
    return (start, duration, value)

In [ ]:
filename = "path_to_greenbutton_xml"


# Existing libraries deal with units and scaling factors within the Green Button data strucutre
data = greenbutton_objects.parse.parse_feed(filename)

# Quick printouts to check what is in each level of data in this structure
mr = list(data[0].meterReadings)[0]
print(mr.intervalBlocks[0].intervalReadings[0].timePeriod.start)
print(mr.intervalBlocks[0])

readings = []

for interval_reading in mr.intervalBlocks[0].intervalReadings:
    readings.append(parse_reading(interval_reading))

# Create dataframe from parsed data in Wh
df = pd.DataFrame(readings,columns=['Start Time','Duration','Wh'])
df = df.set_index('Start Time')

2022-09-01 00:00:00+00:00
<IntervalBlock (Interval Block)>


In [25]:
# Want one calendar year of data for PySAM run - choose 2023 based on the example file we had
df_23 = df.loc['2022-12-31':'2024-01-01']

# The example xml had some missing data so fill it with the nearest value
idx = pd.period_range(min(df_23.index), max(df_23.index), freq='15min')
df_23 = df_23.reindex(idx.astype('datetime64[ns, UTC]'), method='nearest')
print(df_23.loc['2022-12-31':'2024-01-01'])

# Convert Wh to kW (This equation only works for 15 minute data)
kW_data = df_23.loc['2023-01-01 07:00:00':'2024-01-01 06:59:00']['Wh'] / 250.0
print(len(kW_data))
kW_data.to_csv('15_min_load.csv')

                                 Duration    Wh
2022-12-31 00:00:00+00:00 0 days 00:15:00  1036
2022-12-31 00:15:00+00:00 0 days 00:15:00   639
2022-12-31 00:30:00+00:00 0 days 00:15:00   176
2022-12-31 00:45:00+00:00 0 days 00:15:00   192
2022-12-31 01:00:00+00:00 0 days 00:15:00   157
...                                   ...   ...
2024-01-01 22:45:00+00:00 0 days 00:15:00    77
2024-01-01 23:00:00+00:00 0 days 00:15:00   104
2024-01-01 23:15:00+00:00 0 days 00:15:00    78
2024-01-01 23:30:00+00:00 0 days 00:15:00   132
2024-01-01 23:45:00+00:00 0 days 00:15:00   114

[35232 rows x 2 columns]
35040


In [26]:
# Optional: example conversion from 15 minute data in Wh to hourly data in kW

hourly_data = [0] * 8760
for i, v in enumerate(df_23.loc['2023-01-01 07:00:00':'2024-01-01 06:59:00']['Wh'].values):
    hourly_data[int(i / 4)] += v / 1000.0

idx = range(0, 8760)
df_hrly = pd.DataFrame.from_dict({'time' : idx, 'kW' : hourly_data})
df_hrly.to_csv('hourly_load.csv')

In [27]:
# Download rate from URDB and save as file. If rate has already been downloaded, use file
def get_urdb_rate_data(page, key):

    # Full API can be viewed at: https://openei.org/services/doc/rest/util_rates/?version=8
    urdb_url = 'https://api.openei.org/utility_rates?format=json&detail=full&version=8'
    get_url = urdb_url + '&api_key={api_key}&getpage={page_id}'.format(api_key=key, page_id=page)
    print(get_url)

    filename = "urdb_rate_{}.json".format(page)
    print(filename)

    if not os.path.isfile(filename):
        print(get_url)
        resp = requests.get(get_url, verify=certifi.where())
        data = resp.text
        # Cache rate as file
        if "error" not in data:
            with open(filename, 'w') as f:
                f.write(json.dumps(data, sort_keys=True, indent=2, separators=(',', ': ')))
    else:
        with open(filename, 'r') as f:
            data = json.load(f)

    return data

In [ ]:
# Download 15 minute data from NSRDB
building_path = "."
year = 2023

sam_api_key = "YOUR_API_KEY"
sam_email = "YOUR_EMAIL"

nsrdbfetcher = tools.FetchResourceFiles(
                tech='solar',
                nrel_api_key=sam_api_key,
                nrel_api_email=sam_email,
                resource_type='nsrdb-GOES-conus-v4-0-0',
                resource_interval_min=15,
                resource_year=str(year),
                resource_dir=building_path)

lon = "-105.17"
lat = "39.74"

lon_lats = []

lon_lat = (float(lon), float(lat))
lon_lats.append(lon_lat)

nsrdbfetcher.fetch(lon_lats)

# Get rate from URDB
path = os.getcwd() + os.path.sep
page = "67a65130767b93f0b5044b9a"  # https://apps.openei.org/USURDB/rate/view/67a65130767b93f0b5044b9a (residential time of use)

urdb_response = get_urdb_rate_data(page, sam_api_key)
urdb_response_json = json.loads(urdb_response)
if 'error' in urdb_response_json.keys():
    raise Exception(urdb_response_json['error'])
rates = PySAM.UtilityRateTools.URDBv8_to_ElectricityRates(urdb_response_json["items"][0]) 

# Run PV + Battery code with defaults
pv = pv_model.default("PVBatteryResidential") # This runs both PV and battery
ur = utility_rate.from_existing(pv, "PVBatteryResidential")
cl = cashloan.from_existing(ur, "PVBatteryResidential")

for k, v in rates.items():
    try:
        # from_existing above updates this both for PV (battery dispatch) and the utility rates code
        pv.value(k, v)
    except AttributeError:
        if "batt_adjust" in k:
            pass
        else:
            print("Failed to assign PV key " + str(k))

weather_file = "nsrdb_" + lat + "_" + lon + "_nsrdb-GOES-conus-v4-0-0_15_2023.csv"

pv.value("solar_resource_file", str(weather_file))
pv.value("batt_dispatch_choice", 4) # Retail rates dispatch
pv.value("load", kW_data.values)
pv.value("crit_load", kW_data.values) # Not used, but need to set this to 15 minute values to pass checks
pv.value("en_batt", 1)
cl.value("en_batt",1)

pv.execute()
ur.execute()
cl.execute()

# Print out bill savings
print("Year one savings ", ur.Outputs.savings_year1)



Starting data download for solar using 1 thread workers.
File already exists. Skipping download: .\nsrdb_39.74_-105.17_nsrdb-GOES-conus-v4-0-0_15_2023.csv
https://api.openei.org/utility_rates?format=json&detail=full&version=8&api_key=RkdVzEGa7mlEGqV9jzwjhrh95ACZOr6PFr0K0yTc&getpage=67a65130767b93f0b5044b9a
urdb_rate_67a65130767b93f0b5044b9a.json
Year one savings  876.0264276434835
